#  Comparison Between Keras Sequential API and DenseNet API models

In [1]:
import tensorflow
from tensorflow import keras
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras.datasets import mnist

In [3]:
import numpy as np
import pandas as pd

## loading Mnist data

In [4]:
data = mnist.load_data()

In [5]:
X_train, y_train = data[0][0], data[0][1]
X_test, y_test = data[1][0], data[1][1]

## Preparing Data For DenseNet

In [6]:
#The keras densenet model doesn't support image with dimension not less than (32, 32, 1)
X_traind = np.zeros((X_train.shape[0], 50, 50))
X_testd = np.zeros((X_test.shape[0], 50, 50))
for i in range(X_train.shape[0]): X_traind[i] = cv2.resize(X_train[i], (50, 50)) # shape = (60k, 50, 50)
for i in range(X_test.shape[0]): X_testd[i] = cv2.resize(X_test[i], (50, 50)) #shape = (10k, 50, 50)
X_traind = X_traind.astype(np.float16) / 255 
X_testd = X_testd.astype(np.float16) / 255 

X_traind = np.expand_dims(X_traind, -1) #shape = (60k, 50, 50, 1)
X_testd = np.expand_dims(X_testd, -1) #shape = (10k, 50, 50, 1)

y_traind = keras.utils.to_categorical(y_train)
y_testd = keras.utils.to_categorical(y_test)

## Preparing Data For Sequential Model using Convolutions

In [10]:
X_train = X_train.astype(np.float16) / 255 #shape = (60k, 28, 28)
X_test = X_test.astype(np.float16) / 255 #shape = (10k, 28, 28)

X_train = np.expand_dims(X_train, -1) #shape = (60k, 28, 28, 1)
X_test = np.expand_dims(X_test, -1) #shape = (10k, 28, 28, 1)
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

# Sequential Model

In [11]:
model = keras.Sequential()

In [12]:
model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (28, 28, 1)))
model.add(MaxPool2D(4, 4))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPool2D(4, 4))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(10, activation = 'softmax'))

In [13]:
model.compile(optimizer='adam', metrics=['accuracy'], loss=keras.losses.categorical_crossentropy)

In [14]:
model_history_n = model.fit(X_train, y_train, epochs=100, validation_split=0.25)

Epoch 1/100
1407/1407 [==============================] - 14s 10ms/step - loss: 0.7953 - accuracy: 0.7433 - val_loss: 0.1916 - val_accuracy: 0.9510
Epoch 2/100
1407/1407 [==============================] - 13s 10ms/step - loss: 0.3519 - accuracy: 0.8910 - val_loss: 0.1385 - val_accuracy: 0.9602
Epoch 3/100
1407/1407 [==============================] - 14s 10ms/step - loss: 0.2815 - accuracy: 0.9134 - val_loss: 0.1027 - val_accuracy: 0.9711
Epoch 4/100
1407/1407 [==============================] - 13s 10ms/step - loss: 0.2451 - accuracy: 0.9249 - val_loss: 0.0865 - val_accuracy: 0.9744
Epoch 5/100
1128/1407 [=======================>......] - ETA: 2s - loss: 0.2229 - accuracy: 0.9313

In [16]:
plt.plot(model_history_n.history('loss'), label = 'Training Loss')
plt.plot(model_history_n.history('val_loss'), label = 'Validation Loss')
plt.legend()
plt.show()

NameError: name 'model_history_n' is not defined

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model.predict(np.array(([X_test[1]]))) > .99

## DenseNet Model

In [7]:
modeld = keras.applications.DenseNet121(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(50, 50, 1),
    pooling='max',
    classes=10,
)

In [8]:
modeld.compile(optimizer='adam', metrics=['accuracy'], loss=keras.losses.categorical_crossentropy)

In [9]:
modeld.fit(X_traind, y_traind, epochs=50)

Epoch 1/50
  69/1875 [>.............................] - ETA: 19:47 - loss: 0.5822 - accuracy: 0.8188


KeyboardInterrupt



In [ ]:
modeld.evaluate(X_testd, y_testd)

In [ ]:
model.predict(np.array(([X_testd[1]]))) > .99

In [12]:
a = np.loadtxt(r"C:\Users\Subhajit Paul\Pictures\data\pan\pan.csv", delimiter = ",")

In [14]:
a.shape

(424, 512)